In [1]:
from SimilarityClustering import SimilarityClustering
import articles_data

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import cnouns as cn
from pymongo import MongoClient
import datetime

%matplotlib inline

In [2]:
client = MongoClient('mongodb://localhost:27017/somanews')
client.somanews.authenticate('ssomanews', 'ssomanews1029')
db = client.get_database('somanews')

crawled_collection = db.get_collection('crawledArticles')
clusters_collection = db.get_collection('clusters')

In [6]:
catelist_path = '../datastore/category2.p'
# w2v_src_dir = "../datastore/w2v_src"
# w2v_path = "../datastore/sejongcorpus_w2v2.p"

w2v_src_dir = "../datastore/w2v_src3"
w2v_path = "../datastore/sejongcorpus_w2v3_2.p"

corpus_path = "../datastore/corpus2.p"
target_date = datetime.date(2016, 11, 4)
prefix = int("%.2d%.2d"%(target_date.month, target_date.day))
prefix = 1104
prefix_str = "%d_01" % prefix

In [7]:
dicts = [u'새누리', u'새누리당', u'더민주', u'더민주당', u'최순실', u'박대통령', u'국회의장', u'야권의요구', u'정기국회', u'참여정부']
def tokenizer(inp_str):
    return cn.custom_pos_tags(inp_str, dicts)

# Doc2Vec

In [8]:
# size = articles_data.makeDataset(crawled_collection, w2v_src_dir, corpus_path, tokenizer=tokenizer)
# print size

In [9]:
# articles_data.trainWord2Vec(w2v_src_dir, w2v_path, size)

# Model

In [10]:
train_df = articles_data.find_recent_articles(crawled_collection, catelist_path, target_date)
# sc = SimilarityClustering()
# sc.train("cate", w2v_path, train_df, path="../datastore", prefix=prefix_str, tokenizer=tokenizer,
#             threshold=0.8, 
#             cnt_threshold=1, 
#             repeat=1,
#             model_name='dbow+dmm')

In [11]:
train_df.title

0                                  힘받는 국정교과서 철회론…교육감들도 가세
1                   기업 돈 낸 즈음 친기업법안 처리…박 대통령 ‘뇌물죄’ 해석 힘실려
2                    하룻밤 새 50억원 상당 난(蘭) 800여본 사라져 경찰 수사착수
3                          “마음속 ‘분단 철조망’ 걷어내기 프로젝트 이어갈 것”
4                          철도파업 현장 배치 대체인력, 활용 못해 ‘예산 낭비’
5                좋은 일자리 부족이 ‘캥거루족’ 양산…취업해도 절반은 부모에 생활비 의존
6                                이번엔 단풍놀이 버스…휘청하다 갓길서 ‘쾅’
7                              헌재 “스쿠버 장비로 해산물 채취 금지는 합헌”
8                                 “4분 일찍 퇴근 중 숨져도 업무상 재해”
9                                   “7년간 야근하다 돌연사…업무상 재해”
10                   금남로에 시민 3000명…백남기 농민 ‘마지막 길’ 외롭지 않았다
11             [2016 미국의 선택]클린턴 ‘히스패닉 조기투표율 호재’에도…판세 예측불허
12                                      중국발 초미세먼지 한반도 닥치나
13                       [2016 미국의 선택]인지도냐 화려함이냐…‘딸들의 전쟁’
14                                       IS, 극렬한 ‘모술 사수전’
15        [2016 미국의 선택]“아직 모른다” 클린턴은 경합주 단속, 트럼프 민주 텃밭 공략
16                                   11·3 부동산대책에 주택시장 냉기류
17            

In [ ]:
# sc.save(path="../datastore", prefix=prefix_str)

In [ ]:
# sc.d2v_save(path="../datastore", prefix=prefix_str)

In [ ]:
# sc = SimilarityClustering.d2v_load(path="../datastore", prefix=prefix_str)

In [ ]:
sc = SimilarityClustering.load(path="../datastore", prefix=prefix_str, model_name='dbow+dmm', only_d2v=True)

In [ ]:
import deep_utils as du

idx1 = 0
idx2 = 2

# sc.select_model('dbow+dmm')
# v1 = sc.dm_.docvecs[idx1]
# v2 = sc.dm_.docvecs[idx2]
# print du.cs_similarity(v1, v2)

sc.select_model('Doc2Vec(dm/c,d100,n5,w5,mc2,t8)')
v1 = sc.dm_.docvecs[idx1]
v2 = sc.dm_.docvecs[idx2]
print du.cs_similarity(v1, v2)

v_1 = sc.dm_.infer_vector(tokenizer(sc.df_.loc[idx1].title + ' ' + sc.df_.loc[idx1].content))
v_2 = sc.dm_.infer_vector(tokenizer(sc.df_.loc[idx2].title + ' ' + sc.df_.loc[idx2].content))
print du.cs_similarity(v_1, v_2)

In [ ]:
sc.df_[:20].title

In [ ]:
print tokenizer(sc.df_.loc[idx1].title + ' ' + sc.df_.loc[idx1].content)
print '------------------'
print tokenizer(sc.df_.loc[idx2].title + ' ' + sc.df_.loc[idx2].content)

In [ ]:
sc.df_[:5].title

In [ ]:
print sc.df_.loc[idx1].title
print sc.df_.loc[idx2].title

In [ ]:
# sc.select_model('dbow+dmm')
sc.select_model('Doc2Vec(dm/c,d100,n5,w5,mc2,t8)')
sc.cluster_train("cate", path="../datastore", prefix=prefix_str, repeat=1, threshold=0.8)
sc.save(path="../datastore", prefix=prefix_str)

# Save

In [ ]:
asdasd
# sc.save_to_db(prefix, clusters_collection)

# Test

In [ ]:
sc.iner_score(threshold=0.8, cnt_threshold=10)

In [ ]:
sc.print_clusters(top=15, sortby='similarity')

In [ ]:
# sc.print_topics()

In [ ]:
# sc.print_centers()

In [ ]:
# sc.countby_.sort_values('similarity', ascending=False)

In [ ]:
clusters = sc.save_to_db(prefix, clusters_collection)

In [ ]:
# clusters

In [ ]:
from ntc_rank import calc_issue_rank

calced_clusters = calc_issue_rank(clusters)

cdf = pd.DataFrame(calced_clusters, columns=['cluster', 'n', 't', 'c', 'ntc'])
sort_cdf = cdf.sort_values('ntc', ascending=False)
top_cluster = sort_cdf.cluster[1]
sort_cdf

In [ ]:
for idx, row in sort_cdf.iterrows():
    for c in calced_clusters:
        if(c['cluster'] == row.cluster):
            print c['ntc'], c['leading']['title']

In [ ]:
calced_clusters